In [2]:
# Import necessary libs

import pandas as pd
import numpy as np
import os
import re
import random
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

Done


In [5]:
# Randomization
random.seed(3)
pd.options.mode.chained_assignment = None
print ("Turned warnings off")

done


In [8]:
# Read of the data
train_data_2016 = pd.read_csv("../input/train_2016_v2.csv") 
train_data_2017 = pd.read_csv("../input/train_2017.csv")
properties_2016 = pd.read_csv("../input/properties_2016.csv") 
properties_2017 = pd.read_csv("../input/properties_2017.csv")
sample_submission = pd.read_csv("../input/sample_submission.csv")


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Done


In [20]:
# Reading the coloumns with only interger values for simplicity
properties_2016 = properties_2016.select_dtypes(exclude=[object])

# filling in -1 value instead of NaN
properties_2016.fillna(-1,inplace=True)  

# merging the data
train_data_2016 = train_data_2016.loc[:,['parcelid','logerror']].merge(properties_2016,how='left',left_on='parcelid',right_on='parcelid')

# dropping the columns
train_x_2016 = train_data_2016.drop(['parcelid','logerror'],axis=1,inplace=False)

# taking the lag error as the prediction coloumn
train_y_2016 = train_data_2016['logerror']

# taking in the sample file as the test data
test_2016 = sample_submission.loc[:,['ParcelId']].merge(properties_2016,how='left',left_on='ParcelId',right_on='parcelid')
test_x_2016 = test_2016.drop(['ParcelId','parcelid'],axis=1,inplace=False)


In [10]:
# Reading the coloumns with only interger values for simplicity
properties_2017 = properties_2017.select_dtypes(exclude=[object]) 

# filling in -1 value instead of NaN
properties_2017.fillna(-1,inplace=True)

# merging the data
train_data_2017 = train_data_2017.loc[:,['parcelid','logerror']].merge(properties_2017,how='left',left_on='parcelid',right_on='parcelid')

# dropping the columns
train_x_2017 = train_data_2017.drop(['parcelid','logerror'],axis=1,inplace=False)

# taking the log error as the prediction coloumn
train_y_2017 = train_data_2017['logerror']

# taking in the sample file as the test data
test_2017 = sample_submission.loc[:,['ParcelId']].merge(properties_2017,how='left',left_on='ParcelId',right_on='parcelid')
test_x_2017 = test_2017.drop(['ParcelId','parcelid'],axis=1,inplace=False)

In [21]:
# building the random forest refressor model using the sklearn library
model1 = RandomForestRegressor(n_estimators = 200, max_depth=30, max_features= 25 )

# fitting the model
model1.fit(train_x_2016,train_y_2016)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features=25, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [ ]:
# building the random forest refressor model using the sklearn library
model2 = RandomForestRegressor(n_estimators = 200, max_depth=30, max_features= 25 )

model2.fit(train_x_2017,train_y_2017)

In [23]:
pred1 = model1.predict(test_x_2016)

In [24]:
pred2 = model2.predict(test_x_2017)

In [28]:
pred = pd.DataFrame(pred1,columns=['201610'])
pred['201611'] = pred1
pred['201612'] = pred1
pred['201710'] = pred2
pred['201711'] = pred2
pred['201712'] = pred2

submission = pred.copy()
submission["parcelid"] = sample_submission["ParcelId"].copy()
cols = ["parcelid","201610","201611","201612","201710","201711","201712"]
submission = submission[cols]
filename = "PredictedValue.csv"
submission.to_csv(filename,index=False)